In [1]:
import pygame

#load in a font
pygame.font.init()
font = pygame.font.SysFont('comicsans', 75)

SCREEN_TITLE = "Crossing Ninja" 
SCREEN_WIDTH = 400
SCREEN_HEIGHT = 400

#colors can be programmed as RGB codes
WHITE_COLOR = (255,255,255)
BLACK_COLOR = (0,0,0)
BLUE_COLOR = (0,0,150)

clock = pygame.time.Clock()

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class Game:

    TICK_RATE = 60

    def __init__(self,title,width,height):
        self.title=title
        self.width=width
        self.height=height

        #draws the game screen in white with dimensions
        self.game_screen = pygame.display.set_mode((width,height))
        self.game_screen.fill(WHITE_COLOR)
        pygame.display.set_caption(title)
        

    def run_game_loop(self,level):

        game_over = False
        win = False
        lose = False
        direction = 0

        player = PlayerCharacter('player.png', 187, 375, 25, 25)

        enemy = EnemyCharacter('enemy.png', 20, 300, 25, 25)
        enemy.SPEED += level
        enemy1 = EnemyCharacter('enemy.png', self.width - 45, 200, 25, 25)
        enemy1.SPEED += level
        enemy2 = EnemyCharacter('enemy.png', 20, 100, 25, 25)
        enemy2.SPEED += level
        boss = EnemyCharacter('enemy.png', 20, 150, 100, 100)
        boss.SPEED = 2
    
        treasure = GameObject('treasure.png', 187, 10, 25, 25)

        while not game_over:
            for event in pygame.event.get():
                #when it is time to quit the game
                if event.type == pygame.QUIT:
                    game_over = True
                elif event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_UP:
                        direction = 1
                    elif event.key == pygame.K_DOWN:
                        direction = -1
                elif event.type == pygame.KEYUP:
                    if event.key == pygame.K_UP or event.key == pygame.K_DOWN:
                        direction = 0
            self.game_screen.fill(WHITE_COLOR)
            #load, resize and blit the background image onto the gamescreen
            background_image = pygame.image.load('background.png')
            background_image = pygame.transform.scale(background_image,(self.width,self.height))
            self.game_screen.blit(background_image, (0,0))
            #change player position before redrawing it
            player.move(direction, self.height)
            player.draw(self.game_screen)
            #change enemy postion before redrawing it
            if level < 8:
                enemy.move(self.width)
                enemy.draw(self.game_screen)
            if level > 3 and level < 8:
                enemy1.move(self.width)
                enemy1.draw(self.game_screen)
            if level > 5 and level < 8:
                enemy2.move(self.width)
                enemy2.draw(self.game_screen)
            if level > 7:
                boss.move(self.width)
                boss.draw(self.game_screen)
                
            #redraw treasure so it is not covered by the background
            treasure.draw(self.game_screen)

            #detect collision with another object
            if player.detect_collision(enemy) or player.detect_collision(enemy1) or player.detect_collision(enemy2) or player.detect_collision(boss):
                lose = True
                text = font.render('YOU LOSE!',True, BLUE_COLOR)
                self.game_screen.blit(text,(70,175))
                pygame.display.update()
                clock.tick(1)
                break
            if player.detect_collision(treasure):
                win = True
                if level == 8:
                    text1 = font.render('YOU BEAT',True, BLUE_COLOR)
                    self.game_screen.blit(text1,(75,175))
                    text2 = font.render('THE GAME!',True, BLUE_COLOR)
                    self.game_screen.blit(text2, (75,260))
                    text3 = font.render((input ('Enter your name:')),True,BLACK_COLOR)
                    self.game_screen.blit(text3, (20,335))
                    pygame.display.update()
                    clock.tick(1)
                    self.run_game_loop(1)
                text = font.render('YOU WIN!',True, BLUE_COLOR)
                self.game_screen.blit(text,(75,175))
                pygame.display.update()
                clock.tick(1)
                break                   
    
    #update all game graphics
            pygame.display.update()
            clock.tick(self.TICK_RATE)

        if win:
            self.run_game_loop(level + 1)
        if lose:
            self.run_game_loop(level)

In [3]:
#Generic Game Object Class
class GameObject:

    def __init__(self, image_path, x, y, width, height):
        object_image = pygame.image.load(image_path)
                                            #this allows you to dictate the size
                                            #in the code for each object using
                                            #the scaling function
        self.image = pygame.transform.scale(object_image, (width,height))

        self.x_pos = x
        self.y_pos = y

        self.width = width
        self.height = height

    #I wondered if this can be added to the initializer, but it can't because
    #it must be redrawn when it moves
    def draw(self, background):
        background.blit(self.image, (self.x_pos, self.y_pos))

In [4]:
#Player Character Object Class
class PlayerCharacter(GameObject):

    SPEED = 5

    #I wonder if this initializer is necessary; tested it and it doesn't seem to
    #be needed, although I may not know something important
    def __init__(self, image_path, x, y, width, height):
        super().__init__(image_path, x, y, width, height)
    
    def move(self, direction,max_height):
        if direction > 0:
            self.y_pos -= self.SPEED
        elif direction < 0:
            self.y_pos += self.SPEED

        if self.y_pos >= max_height - self.height:
            self.y_pos = max_height - self.height
        if self.y_pos <= max_height - max_height:
            self.y_pos = max_height - max_height

    def detect_collision(self, other_body):
        if self.y_pos + self.height >= other_body.y_pos and self.y_pos <= other_body.y_pos + other_body.height and self.x_pos + self.width >= other_body.x_pos and self.x_pos <= other_body.x_pos + other_body.width:
            return True

In [5]:
#Enemy Character Object Class
class EnemyCharacter(GameObject):

    SPEED = 2

    def __init__(self, image_path, x, y, width, height):
        super().__init__(image_path, x, y, width, height)

    def move(self, max_width):
        if self.x_pos <= 0:
            self.SPEED = abs(self.SPEED)
        elif self.x_pos >= max_width - self.width:
            self.SPEED = -abs(self.SPEED)
        self.x_pos += self.SPEED

In [ ]:
#initialize pygame so that it's functions can be used
pygame.init()

new_game = Game(SCREEN_TITLE,SCREEN_WIDTH,SCREEN_HEIGHT)
new_game.run_game_loop(1) 


#player_image = pygame.image.load('player.png')
#player_image = pygame.transform.scale(player_image, (25,25))
#game_screen.blit(player_image, (200,200))

pygame.quit()
quit